In [1124]:
import pandas as pd
import numpy as np
import os
import variablen as var
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from string import ascii_uppercase
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

In [586]:
filename = var.dataset_name_1
csv_file     = var.original_df_1
save_dir = var.prepair_df_dir
column = 'AMT_INCOME_TOTAL'
result_dir = var.result_dir
statistics_dir = var.statistics_dir
result_file = 'result'

In [ ]:
def split_data(csv_file,filename, column, save_dir):
    df = pd.read_csv(csv_file)
    df = df.dropna() 
    y = df.loc[:, [column]].values
    #y = df.pop(column).to_numpy()
    df = df.drop(columns=[column])
    X = df.loc[: , df.columns].values
    #  X = df.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    
    X_train = pd.DataFrame(X_train, columns= df.columns)
    X_test  = pd.DataFrame(X_test,  columns= df.columns)
    y_train = pd.DataFrame(y_train, columns= [column])
    y_test  = pd.DataFrame(y_test,  columns= [column])
    
    X_train.to_csv( save_dir  + column + '/X_train_' + filename + '.csv'  ,index=False,sep=';', decimal=',')
    X_test.to_csv(  save_dir  + column + '/X_test_'  + filename + '.csv'  ,index=False,sep=';', decimal=',')
    y_train.to_csv( save_dir  + column + '/y_train_' + filename + '.csv'  ,index=False,sep=';', decimal=',')
    y_test.to_csv(  save_dir  + column + '/y_test_'  + filename + '.csv'  ,index=False,sep=';', decimal=',')

In [ ]:
split_data(csv_file,filename, column)

In [ ]:
# Quelle https://rmisstastic.netlify.app/how-to/python/generate_html/how%20to%20generate%20missing%20values
def ampute_mcar(X_complete, missing_rate):
    # Mask completly at random some values
    M = np.random.binomial(1, missing_rate, size = X_complete.shape)
    X_obs = X_complete.copy()
    np.putmask(X_obs, M, np.nan)
    print('Percentage of newly generated mising values: {}'.\
      #Change Code:     
      format(np.round(np.sum(pd.isna(X_obs))/X_obs.size,3)))
    
    # warning if a full row is missing
    for row in X_obs:
        if np.all(pd.isna(row)):
            warnings.warn('Some row(s) contains only nan values.')
            break

    # warning if a full col is missing
    for col in X_obs.T:
        if np.all(pd.isna(col)):
            warnings.warn('Some col(s) contains only nan values.')
            break
            
    return X_obs

In [ ]:
def ampute(column_set):
    for column in column_set:
        for s in var.test_set: 
            X_complete = pd.read_csv(save_dir + column + '/' + s + '.csv' , delimiter=";", decimal=",")
            for missing_rate in var.missing_rats:
                X_obs_mcar =  pd.DataFrame(ampute_mcar(X_complete.to_numpy(), missing_rate))
                X_obs_mcar.columns = [X_complete.columns]
                filename = str(np.int_(100 * missing_rate))
                X_obs_mcar.to_csv(save_dir + column + '/' + s + '/' + filename + ".csv" ,index=False,sep=';', decimal=',')

In [ ]:
ampute(var.application_record)

In [ ]:
def means_impute(column_set):

    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        wert = y_train[column].mean()
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        for i in y_files:
            df = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            results[i[0:-4]] = df.replace(df,wert)
        results.to_csv( result_dir  + var.methode_1 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [ ]:
def mode_impute(column_set):

    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        wert = y_train[column].mode().iloc[0,]
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        for i in y_files:
            df = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            results[i[0:-4]] = df.replace(df,wert)
        results.to_csv( result_dir  + var.methode_2 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [ ]:
def median_impute(column_set):

    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        wert = y_train[column].median()
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        for i in y_files:
            df = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            results[i[0:-4]] = df.replace(df,wert)
        results.to_csv( result_dir  + var.methode_3 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [1026]:
def transform(df,trans_df):
    for column in df.columns:
        changed = pd.Series(data=-1, index=range(df[column].size))
        
        for i, v in trans_df.iterrows():
            change = df[column].ne(v[column])
            df[column] = df[column].where(change, i)
            changed = changed.where(change,1)
        change = changed.ne(-1)
        df[column] = df[column].where(change, -1)
        df[column] = df[column].astype('float64')
    return df

In [1114]:
def retransform(df,trans_df, types):
    for column in df.columns:
        
        for i, v in trans_df.iterrows():
            change = df[column].ne(i)
            df[column] = df[column].where(change,v[column])
        change = df[column].ne(-1)
        df[column] = df[column].astype(types.loc[column])
        df[column] = df[column].where(change, np.nan)
    return df

In [ ]:
result = pd.DataFrame(np.random.randint(100, size=(5, 5)), columns = list("ABCDE"), 
        s = df[[column]].sort_values(by=column, ascending=True).drop_duplicates(ignore_index=True)
        s = s.as

In [890]:
def create_trans_table(df):
    result = pd.DataFrame()
    for column in df.columns:
        s = df[[column]].sort_values(by=column, ascending=True).drop_duplicates(ignore_index=True)
        result = pd.concat([result,s], axis=1)
            
        #s = df[[column]]
        #s.to_csv(var.prepair_df_dir + col + '/columns/' + column  ,index=False,sep=';', decimal=',',  float_format='%.2f')
        #s = pd.read_csv(var.prepair_df_dir  + col + '/columns/' + column , delimiter=";", decimal=",")
        #result[column] = s[column]
    return result

In [587]:
def correlation(dataset,column):
    print(type(column))
    print(column)
    corr = dataset.corrwith(column)
    print(corr)

In [662]:
def correlation(df,column):
    corr = df.corr().abs().unstack(level=0)
    result = corr[column].drop(index=column)
    return result.idxmax()

In [ ]:
print(corr[column].max(level=0))
    kot = corr[corr>=.9]
    plt.figure(figsize=(12,8))
    sns.heatmap(kot, cmap="Greens")

In [581]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

In [582]:
def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

In [ ]:
def correlation_1_impute(column_set):
    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        df = X_train.copy()
        df[y_train.columns] = y_train
        df =  df.drop(columns=['ID'])

        trans_df = create_trans_table(df)
        types = df.dtypes 
        df_temp = transform(df.copy(), trans_df )
        #############################################
        col_corr = correlation(df_temp ,y_train.columns[0])
        # modi_df DF mit Ziel und Corr Spalte
        modi_df = df_temp[[col_corr]] 
        modi_df[y_train.columns[0]] = df[y_train.columns[0]]
        modi_df = modi_df.groupby(col_corr)[[y_train.columns[0]]].median()
        # hinzufügen Missing Bedingung
        df2 =pd.DataFrame([[np.nan, np.nan]], columns=[col_corr, y_train.columns[0]])
        modi_df = modi_df.append(df2, ignore_index=True)
        
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        
        for i in y_files:
            df_y = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            df_x = pd.read_csv(save_dir + column + '/' + var.x_test + '/' + i , delimiter=";", decimal=",")
             
            df_x = df_x.drop(columns=['ID'])  
            x_test = transform(df_x, trans_df )

            for i, v in modi_df.iterrows():
                change = x_test[col_corr].ne(v[col_corr])
                y_test[column] = y_test[column].where(change,v[column])
                results[i[0:-4]] = retransform(y_test, trans_df, types)
        results.to_csv( result_dir  + var.methode_4 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [ ]:
correlation_1_impute(var.application_record)

In [1122]:
def rondom_forest_classification_impute(column_set):
    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        X_train = X_train.drop(columns=['ID'])
        df = X_train.copy()
        df[y_train.columns] = y_train
        trans_df = create_trans_table(df)
        types = df.dtypes 
        x_temp = transform(X_train.copy(), trans_df )
        y_temp = transform(y_train.copy(), trans_df )
        ##############################################
        sc = StandardScaler()
        x_temp = sc.fit_transform(x_temp)
        x_temp_test = sc.transform(x_temp)
        #############################################
        best_tree = -1
        best_mean_squared_error = -1
        ##############################################
        #for i in [40,50,60,70,80,90,100,110,120,130,140,150,160,170]:
        for i in [110]:
            ###################################
            classifier = RandomForestClassifier(n_estimators=i, random_state=0)
            classifier.fit(x_temp, y_temp.ravel())
            y_pred = classifier.predict(x_temp_test)
            ###################################
            new_mean_squared_error = metrics.mean_squared_error(y_temp, y_pred)
            print(str(i) + " durch")
            if best_tree == -1 or new_mean_squared_error < best_mean_squared_error:
                best_mean_squared_error = new_mean_squared_error
                best_tree = i 
                print(best_tree)
                print(best_mean_squared_error)
                print("Best Tree " + str(best_tree))
        
        classifier = RandomForestClassifier(n_estimators=best_tree, random_state=0)
        classifier.fit(x_temp, y_temp.ravel())
        ######################################
        ######################################
        results = y_test[[column]].copy()
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        for i in y_files:
            df_y = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            df_x = pd.read_csv(save_dir + column + '/' + var.x_test + '/' + i , delimiter=";", decimal=",")
            
            df_x = df_x.drop(columns=['ID'])
            x_test = transform(df_x, trans_df )
            X_test = sc.transform(x_test)
            y_pred =  classifier.predict(X_test)
            y_test = y_pred
            results[i[0:-4]] = retransform(y_test, trans_df, types)
        results.to_csv( result_dir  + var.methode_7 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')      

In [ ]:
rondom_forest_classification_impute(var.application_record)

In [ ]:
def rondom_forest_regression_impute(column_set):
    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        X_train = X_train.drop(columns=['ID'])
        df = X_train.copy()
        df[y_train.columns] = y_train
        trans_df = create_trans_table(df)
        types = df.dtypes 
        x_temp = transform(X_train.copy(), trans_df )
        y_temp = transform(y_train.copy(), trans_df )
        ##############################################
        sc = StandardScaler()
        x_temp = sc.fit_transform(x_temp)
        x_temp_test = sc.transform(x_temp)
        #############################################
        best_tree = -1
        best_mean_squared_error = -1
        ##############################################
        #for i in [40,50,60,70,80,90,100,110,120,130,140,150,160,170]:
        for i in [110]:
            ###################################
            classifier = RandomForestClassifier(n_estimators=i, random_state=0)
            classifier.fit(x_temp, y_temp.ravel())
            y_pred = classifier.predict(x_temp_test)
            ###################################
            new_mean_squared_error = metrics.mean_squared_error(y_temp, y_pred)
            print(str(i) + " durch")
            if best_tree == -1 or new_mean_squared_error < best_mean_squared_error:
                best_mean_squared_error = new_mean_squared_error
                best_tree = i 
                print(best_tree)
                print(best_mean_squared_error)
                print("Best Tree " + str(best_tree))
        
        classifier = RandomForestClassifier(n_estimators=best_tree, random_state=0)
        classifier.fit(x_temp, y_temp.ravel())
        ######################################
        ######################################
        results = y_test[[column]].copy()
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        for i in y_files:
            df_y = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            df_x = pd.read_csv(save_dir + column + '/' + var.x_test + '/' + i , delimiter=";", decimal=",")
            
            df_x = df_x.drop(columns=['ID'])
            x_test = transform(df_x, trans_df )
            X_test = sc.transform(x_test)
            y_pred =  classifier.predict(X_test)
            y_test = y_pred
            results[i[0:-4]] = retransform(y_test, trans_df, types)
        results.to_csv( result_dir  + var.methode_8 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')      

In [ ]:
rondom_forest_regression_impute(var.application_record)

In [ ]:

        s = trans_df[col_corr]
        for i, v in s.items():
            new_df = df[df[col_corr] == i ]
            mod = new_df[y_train.columns[0]].mode().iloc[0]
            df2 =pd.DataFrame([[i, mod]], columns=list('IM'))
            modi_df = modi_df.append(df2)
        df2 =pd.DataFrame([[i, mod]], columns=list('IM'))
        y_train.columns[0]
            
            
        #Mode gibt eine Serie's wieder
        #print(new_df.head())
        if not new_df[column].mode().empty:
            new_df[column].fillna(new_df[column].mode().iloc[0], inplace=True)       
        df.update(new_df)

In [ ]:
for methode in var.methodes:
    files = os.listdir(result_dir + methode+ '/' )
    for file in files:
        org_column = file[0:-4]
        result = pd.read_csv(result_dir + methode+ '/' +  file , delimiter=";", decimal=",")
        if os.path.exists(statistics_dir + 'statistics.csv'):
            statistics = pd.read_csv(statistics_dir + 'statistics.csv' , delimiter=";", decimal=",")
        else:
            statistics = pd.DataFrame()
        statistic = pd.DataFrame()
        original = result.loc[: , org_column]
        result = result.drop(columns=[org_column])
        columns = np.array(['methode','type','column', 'misssing', 'Wert'])

        
        for column in result.columns:
            values = np.array([methode,'mean_absolute_error', org_column])
            values = np.append(values, [column])
            values = np.append(values , [metrics.mean_absolute_error(original, result[column])])
            statistics = pd.DataFrame(np.array([values]), columns=columns).append(statistics, ignore_index=True)

        statistics.to_csv(statistics_dir + 'statistics.csv', index=False,sep=';', decimal=',')

In [ ]:
for methode in var.methodes:
    files = os.listdir(result_dir + methode+ '/' )
    for file in files:
        org_column = file[0:-4]
        result = pd.read_csv(result_dir + methode+ '/' +  file , delimiter=";", decimal=",")
        if os.path.exists(statistics_dir + 'statistics.csv'):
            statistics = pd.read_csv(statistics_dir + 'statistics.csv' , delimiter=";", decimal=",")

        else:
            statistics = pd.DataFrame()
            
        original = result.loc[: , org_column]
        result = result.drop(columns=[org_column])
        columns = np.array(['methode','type','column', 'misssing', 'Wert'])

        

        for column in result.columns:
            values = np.array([methode,'mean_squared_error', org_column])
            values = np.append(values, [column])
            values = np.append(values , [metrics.mean_squared_error(original, result[column])])
            statistics = pd.DataFrame(np.array([values]), columns=columns).append(statistics, ignore_index=True)

        statistics.to_csv(statistics_dir + 'statistics.csv', index=False,sep=';', decimal=',')

In [ ]:
for methode in var.methodes:
    files = os.listdir(result_dir + methode+ '/' )
    for file in files:
        org_column = file[0:-4]
        result = pd.read_csv(result_dir + methode+ '/' +  file , delimiter=";", decimal=",")
        if os.path.exists(statistics_dir + 'statistics.csv'):
            statistics = pd.read_csv(statistics_dir + 'statistics.csv' , delimiter=";", decimal=",")
        else:
            statistics = pd.DataFrame()
            
        original = result.loc[: , org_column]
        result = result.drop(columns=[org_column])
        columns = np.array(['methode','type','column', 'misssing', 'Wert'])


        for column in result.columns:
            values = np.array([methode,'root_mean_squared_error', org_column])
            values = np.append(values, [column])
            values = np.append(values , [np.sqrt(metrics.mean_squared_error(original, result[column]))])
            statistics = pd.DataFrame(np.array([values]), columns=columns).append(statistics, ignore_index=True)
        

        statistics.to_csv(statistics_dir + 'statistics.csv', index=False,sep=';', decimal=',')

In [1108]:
df = pd.DataFrame(np.random.randint(100, size=(5, 5)), columns = list("ABCDE"), 
                  index = [i for i in range(5)])
print(df.head())

    A   B   C   D   E
0  58  67  94  16  29
1  75  67  53  50  69
2   1  48  99  73  49
3  63  64  53   5  87
4   0  54  34  83  18


In [1109]:
print(df.dtypes)

A    int32
B    int32
C    int32
D    int32
E    int32
dtype: object


In [1097]:
trans_df = create_trans_table(df)
print(trans_df)

    A   B   C   D   E
0  40  18   7   9  18
1  62  21  35  21  20
2  68  73  37  23  33
3  74  88  54  51  45
4  97  97  95  87  52


In [1098]:
df2 = pd.DataFrame([[101, 102,np.nan,104,105]], columns=list('ABCDE'))
df = df.append(df2, ignore_index=True)
print(df)

     A    B     C    D    E
0   97   18  54.0   23   45
1   68   21  35.0    9   52
2   40   73   7.0   87   20
3   62   88  37.0   51   33
4   74   97  95.0   21   18
5  101  102   NaN  104  105


In [1099]:
df1 = transform(df.copy(), trans_df )
print(df1)

     A    B    C    D    E
0  4.0  0.0  3.0  2.0  3.0
1  2.0  1.0  1.0  0.0  4.0
2  0.0  2.0  0.0  4.0  1.0
3  1.0  3.0  2.0  3.0  2.0
4  3.0  4.0  4.0  1.0  0.0
5 -1.0 -1.0 -1.0 -1.0 -1.0


In [1115]:
df2 = retransform(df1.copy(), trans_df, df.dtypes)
print(df2)

      A     B     C     D     E
0  97.0  18.0  54.0  23.0  45.0
1  68.0  21.0  35.0   9.0  52.0
2  40.0  73.0   7.0  87.0  20.0
3  62.0  88.0  37.0  51.0  33.0
4  74.0  97.0  95.0  21.0  18.0
5   NaN   NaN   NaN   NaN   NaN


In [1107]:
print(df2.dtypes)

A    object
B    object
C    object
D    object
E    object
dtype: object


In [1112]:
df2 = df2.convert_dtypes()

In [1113]:
print(df2.dtypes)

A    object
B    object
C    object
D    object
E    object
dtype: object


In [1045]:
print(df.dtypes)

A      int64
B      int64
C    float64
D      int64
E      int64
dtype: object


In [863]:
changed = pd.Series(data=-1, index=range(df['A'].size))
print(changed)

0   -1
1   -1
2   -1
3   -1
4   -1
dtype: int64


In [874]:
v = pd.Series(range(8))


change = df['C'].eq(v)
print(change)

0     True
1    False
2    False
3    False
4     True
5    False
6    False
7    False
dtype: bool


In [817]:
df['C'] = df['C'].where(change, "5")
print(df)

   A  B  C  D    E
0  2  1  5  4  2.0
1  3  3  1  1  1.0
2  4  2  5  3  3.0
3  0  0  3  2  4.0
4  1  4  4  0  0.0


In [876]:
df = pd.concat([df,v], axis=1)
print(df)

     A    B    C    D    E  0
0  3.0  0.0  0.0  2.0  1.0  0
1  0.0  3.0  3.0  0.0  3.0  1
2  1.0  1.0  1.0  4.0  2.0  2
3  2.0  4.0  2.0  1.0  0.0  3
4  4.0  2.0  4.0  3.0  4.0  4
5  NaN  NaN  NaN  NaN  NaN  5
6  NaN  NaN  NaN  NaN  NaN  6
7  NaN  NaN  NaN  NaN  NaN  7


In [853]:
print(df)

Index


In [848]:
print(df.columns)

Index(['A', 'B', 'C', 'D', 'E'], dtype='object')


In [808]:
changed = changed.where(change, 1)
print(changed)

0    1
1   -1
2    1
3    1
4   -1
dtype: int64


In [809]:
m = changed.eq(-1)

In [1024]:
df = pd.DataFrame({'float': [1.0],
                   'int': [1],
                   'datetime': [pd.Timestamp('20180310')],
                   'string': ['foo']})
types = df.dtypes
print(types.head())
print(types.loc['int'])

float              float64
int                  int64
datetime    datetime64[ns]
string              object
dtype: object
int64


In [1086]:
df = pd.DataFrame([["", pd.NA, np.nan, "3"]])
print(df)

  0     1   2  3
0    <NA> NaN  3


In [1085]:
print(df.dtypes)

0     object
1     object
2    float64
3     object
dtype: object


In [1093]:
df = df[3].astype('int64')

In [1094]:
print(df.dtypes)

int64
